##### Next update of this network contains:
- normal costs
- proper nodes at one-way bridges, adding bridges to network

### Update
- Animation function; animator nog niet gerund dus niet een zwaar notebook
- Verschillende classes
- Angle update; previous; r=angle calculated in radians, converted to degrees, and than again for calculation converted back to radians. This gives rounding errors, therefore angle is now all calculated in radians
- Class update; Cruise_shipping_vessel -> Liefst leest hij dit af van een excel, dan kan je dat alter nog veranderen!!



In [2]:
%load_ext autoreload
%autoreload 2
import random
import geopandas as gpd
import networkx as nx
#import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
from shapely.geometry import Point
from matplotlib import animation
from JSAnimation import IPython_display
%matplotlib inline

import matplotlib.pylab as plt





In [3]:
#Show with attributes 
graph = gpd.read_file('/Users/JeroenvanderDoes/Dropbox/Studie/Master/Thesis/Onderwerpen/Programs/QGIS/adjusted_complete_graph/edges_water_2.shp')
print(graph.iloc[3]['cost'])
print(graph.head()) # see what attrinutes the edges and nodes from the shapefile contain

6.68129704879482
         cost  source  target  source_aan  target_aan      width  \
0    2.483749     5.0     3.0         3.0         3.0  24.797941   
1  101.826942     5.0   254.0         3.0         3.0  17.942054   
2   11.685232   212.0   217.0         3.0         3.0  10.357033   
3    6.681297   212.0   215.0         3.0         3.0   4.825559   
4    5.221776   215.0   213.0         3.0         3.0   4.365561   

                                            geometry  
0  LINESTRING (120005.5486256767 487357.479141514...  
1  LINESTRING (120005.5486256767 487357.479141514...  
2  LINESTRING (120071.6506543915 487249.971786562...  
3  LINESTRING (120071.6506543915 487249.971786562...  
4  LINESTRING (120074.1340182582 487243.773574981...  


In [ ]:
G=nx.read_shp('/Users/JeroenvanderDoes/Dropbox/Studie/Master/Thesis/Onderwerpen/Programs/QGIS/adjusted_complete_graph/edges_water_2.shp', simplify=True) # use simplify is true otherwise chart get shifted
w_G = nx.Graph()

color_map = []
lat_lon_to_index = {}

for i, node in enumerate(G.nodes()): #adds nodes with label 1 till n nodes to the network. 
    w_G.add_node(i, pos= node)
    lat_lon_to_index[node] = i
    color_map.append('b')
    
for edge in G.edges(data=True): # adds edges with attributes to the graph from shapefile according to the order in which the edges are ordered in the shapefile
    w_G.add_edge(lat_lon_to_index[edge[0]], lat_lon_to_index[edge[1]], dis=edge[2]['cost'] )



In [ ]:
#nodelist omzetten naar pandas dataframe omdat zodat hij veel makkelijker en sneller de coordinaten van de nodes kan opzoeken
#pandas >>> nx.get_node_attributes kwa snelheid
graph_df = pd.DataFrame.from_dict(nx.get_node_attributes(w_G, 'pos'), orient='index', columns=["lat", "long"]) 

In [ ]:
graph_df.head()

In [ ]:
distance = ((nx.get_node_attributes(w_G, 'pos')[4091][0]-nx.get_node_attributes(w_G, 'pos')[4013][0])**2+(nx.get_node_attributes(w_G, 'pos')[4091][1]-nx.get_node_attributes(w_G, 'pos')[4013][1])**2)**0.5
print(distance)
# check if this is correct with distance measured in maps and one that is attribute on the egde

In [ ]:
i =nx.all_simple_paths(w_G,random.sample(w_G.nodes, 1)[0],random.sample(w_G.nodes, 1)[0], cutoff=None)
print(i)

### Vessel base class

In [ ]:
class Vessel(object):
        """
    Baseclass Vessel kan worden overschreven door nieuwe __init__ aan te maken. 
    Dan moet je wel alle waardes opnieuw bepalen, dus zorg dat je goed copy pasted en alleen de waardes veranderd die
    je wilt veranderen. In dit geval dat het start en eind punt dus bepaald zijn, en dus niet meer als input moet worden gegeven
    omdat het pad al bepaald is. 
    """
    def __init__(self, start, end):
        self.start = start#; Moeten in child classes wel worden gedefinieerd, anders doen deze schepen het niet.
        self.end = end
        self.path = self.route(w_G,self.start, self.end)
        self.coor_log = []
        self.vessel_pos = nx.get_node_attributes(w_G, 'pos')[self.path[0]] 
        self.w_G = w_G
        self.to_node_id = 1
        self.move = 3.0 #aanpassen vooor de tijdsstap, nu  in m/s, en pas aan voor overshoot. 
        self.travel_time = 0
        self.active = True
        self.coor_log_x =[]
        self.coor_log_y=[]
        #self.start_time = simulation.step_count
        
        
        
    def route(self, w_G, start, end):
        return nx.dijkstra_path(w_G, start, end, weight = 'dis')
    
    
    def Distance(self, Node1, Node2, Print=False):
        # calculate distance between two points
        distance = float(((Node2[0]-Node1[0])**2+(Node2[1]-Node1[1])**2)**0.5)
        return float(distance)
    

    def Angle(self, Node1, Node2, Print=False):
        # calculate angle going from point 1 to point 2 (NB: atan2(y,x))
        angle =  (float(math.atan2(Node2[1]-Node1[1],Node2[0]-Node1[0]))) #*float((180.0 / math.pi)) for degrees
        return float(angle)
        

    def Move(self, Node1, Node2, move, Print=False):#move cannot be replaced by self.move because move may change
        # move towards Node2
        distance = self.Distance(Node1, Node2) #wordt dit wel hier gebruikt?
        angle = self.Angle(Node1, Node2)
        new_node = (float(Node1[0]+math.cos(angle)*move), float(Node1[1]+math.sin(angle)*move))
        return new_node

    
    
    def Move_on_path(self, Print=False):
        # 1. vessel_pos is your boat
        # 2. you need to know the next node in the path
        # 3. calculate the distance between Node1 and the next node
        
        if self.active == True: #start logging  each step of vessel;
            
            to_node_pos = (graph_df['lat'].loc[self.path[self.to_node_id]], graph_df['long'].loc[self.path[self.to_node_id]])

            distance_to_next_node = self.Distance(self.vessel_pos, to_node_pos, True)
        

            # If move <= distance to next node, make move
            
            if self.move < distance_to_next_node:
                    vessel_pos_new = self.Move(self.vessel_pos, to_node_pos, self.move, True)
                    self.travel_time+=1

            else:
                if self.to_node_id == len(self.path)-1:
                    self.active = False #stop logging if vessel arrives at destination
                    vessel_pos_new = to_node_pos  
                    self.travel_time +=1
                else:
                    # reposition vessel at next node and calculate remaining move distance
                    self.vessel_pos = (graph_df['lat'].loc[self.path[self.to_node_id]], graph_df['long'].loc[self.path[self.to_node_id]])

                    # up the to_node_id with 1 and find new to_node_pos
                    self.to_node_id+=1
                    
                    # calculate distance to next node
                    
                    remaining_move = float(self.move-distance_to_next_node)
                    
                    #to_node_pos = (graph_df['lat'].loc[self.path[self.to_node_id]], graph_df['long'].loc[self.path[self.to_node_id]])
                    #distance_to_next_node = self.Distance(self.vessel_pos, to_node_pos, True)
                    
                    """
                    if remaining_move < distance_to_next_node:
                        vessel_pos_new = self.Move(self.vessel_pos, to_node_pos, remaining_move, True)
                        self.travel_time+= 1
                        
                    else:
                        self.vessel_pos = (graph_df['lat'].loc[self.path[self.to_node_id]], graph_df['long'].loc[self.path[self.to_node_id]])
                        self.to_node_id+=1
                        to_node_pos = (graph_df['lat'].loc[self.path[self.to_node_id]], graph_df['long'].loc[self.path[self.to_node_id]])
                        distance_to_next_node = self.Distance(self.vessel_pos, to_node_pos, True)
                        remaining_move = remaining_move - distance_to_next_node
                        
                        vessel_pos_new = self.Move(self.vessel_pos, to_node_pos, remaining_move, True)
                        self.travel_time+= 1
                    """    
                    vessel_pos_new = self.Move(self.vessel_pos, to_node_pos, remaining_move, True)
                    self.travel_time+= 1
                        
                   

            # If move > distance to next node, move to next node and set move to move-distance, try again
            if Print == True:
                print('vessel_pos_new:', vessel_pos_new)
                print('to_node_id :', self.to_node_id)
                print('')

            #self.coor_log.append((self.time, {"pos":vessel_pos_new})) # w_G.add.node()labled as the way networkx labels nodes
            self.coor_log.append(vessel_pos_new) #library for tuple (x,y) coordinate of the vessel
            self.coor_log_x.append(vessel_pos_new[0]) # library for only x coordinates
            self.coor_log_y.append(vessel_pos_new[1])# library for only y coordinates
            self.vessel_pos = vessel_pos_new

            #return vessel_pos_new; for updating the simulator class?

In [ ]:
class Cruise_Vessel(Vessel):

     #TODO: is het niet beter baseclass aan te passen zodat er geen start, end is gedefinieerd?
    def __init__(self):
        self.start = 2076 #begin path / read from excel file?
        self.end = 773 #/ #/ read from excel file?
        self.path = self.route(w_G,self.start, self.end)
        self.coor_log = []
        self.vessel_pos = nx.get_node_attributes(w_G, 'pos')[self.path[0]] 
        self.w_G = w_G
        self.to_node_id = 1
        self.move = 3.0 #aanpassen vooor de tijdsstap, nu  in m/s, en pas aan voor overshoot. 
        self.travel_time = 0
        self.active = True
        self.coor_log_x =[]
        self.coor_log_y=[]

    #def route(self, w_G, start, end):
     #   tour_path=[]
      #  h =nx.dijkstra_path(w_G, start, end, weight = 'dis')
        #p =nx.dijkstra_path(w_G, 980, 3806, weight = 'dis')
       #    tour_path.append(i)

        #for j in p:
         #   tour_path.append(j)

        #return tour_path

    #def starttime? Of moet die in de simulator class worden gedaan?
    
    
    
    

In [ ]:
class Random_Cruise_Vessel(Vessel):

     #TODO: is het niet beter baseclass aan te passen zodat er geen start, end is gedefinieerd?
    def __init__(self):
        self.path = self.route(w_G,random.sample(w_G.nodes, 1)[0], random.sample(w_G.nodes, 1)[0])
        self.start = self.path[0]
        self.end = self.path[len(self.path)-1]
        self.coor_log = []
        self.vessel_pos = nx.get_node_attributes(w_G, 'pos')[self.path[0]] 
        self.w_G = w_G
        self.to_node_id = 1
        self.move = 3.0 #aanpassen vooor de tijdsstap, nu  in m/s, en pas aan voor overshoot. 
        self.travel_time = 0
        self.active = True
        self.coor_log_x =[]
        self.coor_log_y=[]
  
    def route(self, w_G, start, end):
        pass
        
    #insert logica for touristiness certain edges
    
    
    
    

In [ ]:
class Pleasure_Craft(Vessel):
    """
    Baseclass Vessel kan worden overschreven door nieuwe __init__ aan te maken. 
    Dan moet je wel alle waardes opnieuw bepalen, dus zorg dat je goed copy pasted en alleen de waardes veranderd die
    je wilt veranderen. In dit geval dat het start en eind punt dus bepaald zijn, en dus niet meer als input moet worden gegeven
    omdat het pad al bepaald is. 
    """
    #TODO: is het niet beter baseclass aan te passen zodat er geen start, end is gedefinieerd?
    def __init__(self):
        self.path = self.route(w_G,random.sample(w_G.nodes, 1)[0], random.sample(w_G.nodes, 1)[0])
        self.start = self.path[0]
        self.end = self.path[len(self.path)-1]
        self.coor_log = []
        self.vessel_pos = nx.get_node_attributes(w_G, 'pos')[self.path[0]] 
        self.w_G = w_G
        self.to_node_id = 1
        self.move = 3.0 #aanpassen vooor de tijdsstap, nu  in m/s, en pas aan voor overshoot. 
        self.travel_time = 0
        self.active = True
        self.coor_log_x =[]
        self.coor_log_y=[]

        #def route(self, w_G, start, end):
         #   return random.sample(nx.all_simple_paths(w_G, start, end, cutoff=None))
        #nx.all_simple_paths(w_G, start, end, cutoff=None)[0]
     
        
        

In [ ]:
class Simulation():
    def __init__(self):
        self.stepsize = 10
        self.sim_steps = 100 #int((24*60*60)/self.stepsize)
        self.vessel_count = 16
        self.ves_list = [] #list of active vessel
        self.start_time_list = [] # list for starttimes for vessel for pd.dataframe
        self.distr = self.distr_gen() #chance generator for distribution of vessel of a given day, can be modified to represent different scenarios
        self.step_count = 0
        
        
        
    def run_full_sim(self):
        for i in range(self.sim_steps):
            self.one_sim_step()
            print(i)
        #Create pandas dataframe of vessels log' for plotting purposes.
#         self.df = pd.DataFrame(np.zeros([self.sim_steps, len(self.ves_list)]))# generate dataframe based on rows: # steps ; colums: # vessels
        self.x_df = pd.DataFrame(np.zeros([self.sim_steps, len(self.ves_list)]))
        self.y_df = pd.DataFrame(np.zeros([self.sim_steps, len(self.ves_list)]))
        
        for i, time, ves in zip(range(len(self.ves_list)), self.start_time_list, self.ves_list):
            #creates dataframe for x coordinates and one for y coordinates based on vessel logs
            ds_x = pd.Series(ves.coor_log_x)
            ds_y =pd.Series(ves.coor_log_y)
            self.x_df[i].iloc[time : time + len(ds_x)] = ds_x.values
            self.y_df[i].iloc[time : time + len(ds_y)] = ds_y.values
            
            #replaces values of 0 to nan for plotting purposes
            self.x_df = self.x_df.replace(0.0, np.nan) 
            self.y_df = self.y_df.replace(0.0, np.nan)

            """only adds coordinates stamps as column in data frame based on timestep in start_time_list; so if a vessel is generated in timestep 20 and
            logs every step and arrives at timestep 30, the coordinates of the timesteps 20 untill 30 are added to the dataframe
            on row 20 untill 30. """
            
        
    def one_sim_step(self):
        for x in range(self.distr[self.step_count]):# generates list of number of boats to be generated for a timestep
            
            #definieerd verschillende schepen met schillende routes en starttijden.
            
            #self.ves_list.append(Vessel(random.sample(w_G.nodes, 1)[0],random.sample(w_G.nodes, 1)[0]))
            
            #self.ves_list.append(Pleasure_Craft())
            self.ves_list.append(Cruise_Vessel())
            self.start_time_list.append(self.step_count) # Generate list of starttimes whenever a vessel is generated for pd.DataFrame
            
        for ves in self.ves_list:
            ves.Move_on_path()
        
        self.step_count += 1

        
   
    def distr_gen(self): # voeg boot toe met bepaalde kans 
        threshold = self.vessel_count / float(self.sim_steps)
        count_list = []
    
        # maakt lijst met hoeveel boten er spawnen op elke tijdstap
        for i in range(self.sim_steps):
            counter = 0
            
            while True:
                nummer = random.random()
                if nummer < threshold:
                    counter += 1
                else:
                    break
            count_list.append(counter)
        return count_list
    
    

In [ ]:
simulation = Simulation()
simulation.run_full_sim()

In [ ]:
#simulation.x_df #Log of route 

## Animator


In [ ]:
%matplotlib notebook
from matplotlib import animation
from JSAnimation import IPython_display
from IPython.display import HTML

fig = plt.figure(figsize=(20,20))
ax = plt.axes()


nx.draw_networkx_edges(w_G,nx.get_node_attributes(w_G, 'pos'))
n_steps = simulation.x_df.index 

graph, = plt.plot([],[],'o')


def get_data_x(i):
    return simulation.x_df.loc[i]

def get_data_y(i):
    return simulation.y_df.loc[i]


def animate(i):
    x = get_data_x(i)
    y= get_data_y(i)
    graph.set_data(x,y)
    return graph,


animation.FuncAnimation(fig, animate, frames=n_steps, repeat=True)


In [ ]:
%matplotlib notebook
fig = plt.figure(figsize=(10,10))
nx.draw_networkx_edges(w_G,nx.get_node_attributes(w_G, 'pos'))
plt.scatter(simulation.x_df,simulation.y_df)
plt.savefig('/Users/JeroenvanderDoes/Downloads/Amsterdam_Canals_11.pdf', format='pdf', rasterized=False)
plt.show()